# Data Engineer Virtual Internship Program

## Import libraries

In [1]:
import psycopg2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Connecting to PostgreSQL

In [2]:
try:
    connection = psycopg2.connect(
        dbname='KALBE',
        user='postgres',
        password='diversionofthree',
        host='localhost',
        port='5432'
    )
    print('Successfully connected to the database!')
except psycopg2.Error as e:
    print(f"ERROR: {e}")

Successfully connected to the database!


## Show tables from KALBE database

In [3]:
query = 'SELECT * FROM public.inventory;'
df = pd.read_sql_query(query, connection)
df

,item_code,item_name,item_price,item_total
0,2341,Promag Tablet,3000,100
1,2342,Hydro Coco 250ML,7000,20
2,2343,Nutrive Benecol 100ML,20000,30
3,2344,Blackmores Vit C 500Mg,95000,45
4,2345,Entrasol Gold 370G,90000,120


In [4]:
query = 'SELECT * FROM public.customer_orders;'
df = pd.read_sql_query(query, connection)
df

,order_no,purchase_amount,order_date,customer_id,salesman_id
0,10001,150,2022-10-05,2005,3002
1,10009,279,2022-09-10,2001,3005
2,10002,65,2022-10-05,2002,3001
3,10004,110,2022-08-17,2009,3003
4,10007,948,2022-09-10,2005,3002
5,10005,2400,2022-07-27,2007,3001


## Show Item_name that has the highest number in Item_total

In [5]:
query_item_total_high = 'SELECT item_name FROM public.inventory ORDER BY item_total DESC LIMIT 1;'
num_high_item = pd.read_sql_query(query_item_total_high, connection)
num_high_item

,item_name
0,Entrasol Gold 370G


## Update the Item_price of the output of question bullet

In [6]:
num_high_item = pd.read_sql_query(query_item_total_high, connection)
new_item_price = 96900
query_update_item_price = f"UPDATE public.inventory SET item_price = {new_item_price} WHERE item_name = '{num_high_item['item_name'][0]}';"
with connection.cursor() as cursor:
    cursor.execute(query_update_item_price)
connection.commit()
print("Item_price updated successfully.")

Item_price updated successfully.


In [7]:
query = 'SELECT * FROM public.inventory;'
df = pd.read_sql_query(query, connection)
df

,item_code,item_name,item_price,item_total
0,2341,Promag Tablet,3000,100
1,2342,Hydro Coco 250ML,7000,20
2,2343,Nutrive Benecol 100ML,20000,30
3,2344,Blackmores Vit C 500Mg,95000,45
4,2345,Entrasol Gold 370G,96900,120


## What will happen if we insert another Item_name with Item_code of 2343 into the table?

In [8]:
new_item = ("Entrostop", 2343, 50, 19000)
query_insert_item = "INSERT INTO public.inventory (item_name, item_code, item_total, item_price) VALUES (%s, %s, %s, %s);"
with connection.cursor() as cursor:
    cursor.execute(query_insert_item, new_item)
connection.commit()
print("Successfully inserted new items.")


Successfully inserted new items.


In [9]:
query = 'SELECT * FROM public.inventory;'
df = pd.read_sql_query(query, connection)
df

,item_code,item_name,item_price,item_total
0,2341,Promag Tablet,3000,100
1,2342,Hydro Coco 250ML,7000,20
2,2343,Nutrive Benecol 100ML,20000,30
3,2344,Blackmores Vit C 500Mg,95000,45
4,2345,Entrasol Gold 370G,96900,120
5,2343,Entrostop,19000,50


## Delete the Item_name that has the lowest number of Item_total

In [16]:
try:
    query_delete_lowest_item = "DELETE FROM public.inventory WHERE item_total = (SELECT MIN(item_total) FROM public.inventory);"

    with connection.cursor() as cursor:
        cursor.execute(query_delete_lowest_item)

    connection.commit()
    print("Successfully deleted item with lowest Item_total value.")

except psycopg2.Error as e:
    connection.rollback()  # Rollback the transaction to clear the error state
    print(f"Error: {e}")

Successfully deleted item with lowest Item_total value.


In [17]:
query = 'SELECT * FROM public.inventory;'
df = pd.read_sql_query(query, connection)
df

,item_code,item_name,item_price,item_total
0,2341,Promag Tablet,3000,100
1,2343,Nutrive Benecol 100ML,20000,30
2,2344,Blackmores Vit C 500Mg,95000,45
3,2345,Entrasol Gold 370G,96900,120
4,2343,Obat Decolgen,15000,50


## Query Create (Challenge No.5)

Create a Query to display all customer orders where purchase amount is less than 100 or exclude those orders which order date is on or greater than 25 Aug 2022 and customer id is above 2001. Sample table: customer_orders

In [19]:
query_display = "SELECT * FROM customer_orders WHERE (purchase_amount < 100 OR (order_date < '2022-08-25' AND customer_id <= 2001));"

In [21]:
dfdisplay = pd.read_sql_query(query_display, connection)
dfdisplay

,order_no,purchase_amount,order_date,customer_id,salesman_id
0,10002,65,2022-10-05,2002,3001
